In [ ]:
#from train4-noGauss; use cbam
import torch,os 
import torch.nn as nn
import torch.optim as optim

from torchvision.transforms import v2
import torchvision.models as models
import pandas as pd
import matplotlib.pyplot as plt

from nets import ResNet18_3lbCBAM
from utils import progress_bar
from tqdm import tqdm
from configparser import ConfigParser
from torch.utils.data import  DataLoader
from LIDC_data import LIDC_Dataset

In [19]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

prep_tr = [
    v2.Lambda(lambda x: torch.clamp(x,-1000,400)),
    v2.Lambda(lambda x: (x+1000)/1400),
    v2.CenterCrop((384,384)),
    # v2.Lambda(lambda x: x.expand(3,-1,-1))
]
aug_tr = [
    v2.RandomAffine(degrees=10),
    v2.RandomHorizontalFlip(),
    # v2.GaussianNoise(0,0.1)
]
trans_train = v2.Compose( prep_tr + aug_tr )
trans_test = v2.Compose( prep_tr  )

In [20]:

parser = ConfigParser()
parser.read('.settings')
root_dir = parser.get('dataset','root_dir') #/workspaces/data/lidc-idri/slices
meta_dir = parser.get('dataset','meta_dir') #/workspaces/data/lidc-idri/splits
train_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'train_malB.csv'),transform=trans_train)
test_data = LIDC_Dataset(root_dir,metapath=os.path.join(meta_dir,'test_malB.csv'),transform=trans_test)
total_train_data = len(train_data)
total_test_data = len(test_data)
print('total_train_data:',total_train_data, 'total_test_data:',total_test_data)

trainloader = DataLoader(train_data, batch_size=16, shuffle=True)
testloader = DataLoader(test_data, batch_size=16)

total_train_data: 5495 total_test_data: 2354


In [ ]:
net = ResNet18_3lbCBAM()
# net = models.resnet18(pretrained=True)
net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
net.fc = nn.Linear(net.fc.in_features, 2)
net = net.to(device)

In [22]:
lr = 1e-4
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)

trainning_accuracy=[]
trainning_loss=[]
testing_accuracy=[]
testing_loss=[]

In [23]:
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader,desc=f"[Epoch {epoch+1}]")):
        inputs, targets = inputs.to(device), targets.to(device)
        # targets = targets.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = net(inputs)
        
        loss = criterion(outputs, targets)
        loss.backward()
        
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        # preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()
        
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        # correct += (preds == targets.long()).sum().item()

    train_acc = 100.*correct/total
    train_loss = train_loss/(batch_idx+1)
    print(f"Train Loss: {train_loss}, Train Acc: {train_acc:.2f}%")
    trainning_accuracy.append(train_acc)
    trainning_loss.append( train_loss )

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            # targets = targets.float().unsqueeze(1)
            outputs = net(inputs)
            
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            # preds = (torch.sigmoid(outputs) > 0.5).squeeze().long()
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            # correct += (preds == targets.long()).sum().item()

            progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        test_acc = 100.*correct/total
        test_loss = test_loss/(batch_idx+1)
        print(f"Test Loss: {test_loss}, Test Acc: {test_acc:.2f}%")
        testing_accuracy.append(100.*correct/total)
        testing_loss.append(test_loss/(batch_idx+1))
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt_owndata.pth')
        best_acc = acc

In [24]:
for epoch in range(start_epoch, start_epoch+50):
    train(epoch)
    test(epoch)
 
 #   scheduler.step()

[Epoch 1]: 100%|██████████| 344/344 [01:39<00:00,  3.47it/s]


Train Loss: 0.6895412580911503, Train Acc: 57.32%
 [=================================================>]  Step: 39ms | Tot: 24s268ms | Loss: 0.637 | Acc: 64.571% (1520/2354 148/148  
Test Loss: 0.6372088226112159, Test Acc: 64.57%
Saving..


[Epoch 2]: 100%|██████████| 344/344 [01:30<00:00,  3.81it/s]


Train Loss: 0.6534980103200259, Train Acc: 60.93%
 [=================================================>]  Step: 22ms | Tot: 19s920ms | Loss: 0.629 | Acc: 64.189% (1511/2354 148/148  
Test Loss: 0.6293733013240067, Test Acc: 64.19%


[Epoch 3]: 100%|██████████| 344/344 [01:29<00:00,  3.82it/s]


...

Train Loss: 0.09681746571495328, Train Acc: 96.49%
 [=================================================>]  Step: 22ms | Tot: 18s528ms | Loss: 0.521 | Acc: 86.534% (2037/2354 148/148  
Test Loss: 0.5207328694995266, Test Acc: 86.53%


[Epoch 49]: 100%|██████████| 344/344 [01:27<00:00,  3.92it/s]


Train Loss: 0.07191701231491861, Train Acc: 97.27%
 [=================================================>]  Step: 23ms | Tot: 19s144ms | Loss: 0.521 | Acc: 86.576% (2038/2354 148/148  
Test Loss: 0.5210785649125338, Test Acc: 86.58%


[Epoch 50]: 100%|██████████| 344/344 [01:32<00:00,  3.73it/s]


Train Loss: 0.07354717591891152, Train Acc: 97.45%
 [=================================================>]  Step: 30ms | Tot: 24s632ms | Loss: 0.567 | Acc: 84.834% (1997/2354 148/148  >.............................................]  Step: 202ms | Tot: 2s337ms | Loss: 0.616 | Acc: 82.917% (199/240 15/148 
Test Loss: 0.5668940258722098, Test Acc: 84.83%


In [25]:
net

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): CBAMBasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (cbam): CBAM(
        (ca): ChannelAttention(
          (shared_mlp): Sequential(
            (0): Linear(in_features=64, out_features=4, bias=False)
            (1): ReLU()
            (2): Linear(in_features=4, out_features=64, bias=Fal

In [26]:
from torchinfo import summary
summary(net, input_size=(16,1, 384, 384))

Layer (type:depth-idx)                             Output Shape              Param #
ResNet                                             [16, 2]                   --
├─Conv2d: 1-1                                      [16, 64, 192, 192]        3,136
├─BatchNorm2d: 1-2                                 [16, 64, 192, 192]        128
├─ReLU: 1-3                                        [16, 64, 192, 192]        --
├─MaxPool2d: 1-4                                   [16, 64, 96, 96]          --
├─Sequential: 1-5                                  [16, 64, 96, 96]          --
│    └─CBAMBasicBlock: 2-1                         [16, 64, 96, 96]          --
│    │    └─Conv2d: 3-1                            [16, 64, 96, 96]          36,864
│    │    └─BatchNorm2d: 3-2                       [16, 64, 96, 96]          128
│    │    └─ReLU: 3-3                              [16, 64, 96, 96]          --
│    │    └─Conv2d: 3-4                            [16, 64, 96, 96]          36,864
│    │    └─BatchNorm2

In [27]:
# net0 = models.resnet18(pretrained=True)
# net0.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
# net0.fc = nn.Linear(net0.fc.in_features, 2)
# net0 = net0.to(device)
# summary(net0, input_size=(16,1, 384, 384))

In [28]:
net.linear = nn.Linear(73728,2)

In [29]:
net.linear

Linear(in_features=73728, out_features=2, bias=True)